In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.sparse.linalg

In [ ]:
from topo import IntervalTopo1D
from basis import LagrangeBasis1D
from mesh import Mesh1D
from assemble import simple_assembly, simple_build_rhs

In [ ]:
order   = 3
L       = 2*np.pi
n_elems = 20

vertices = np.linspace(0, L, n_elems+1)
elem_to_vertex = np.zeros((n_elems, 2), dtype=np.int)
elem_to_vertex[:,0] = np.arange(n_elems)
elem_to_vertex[:,1] = np.arange(n_elems)+1

topo  = IntervalTopo1D()
basis = LagrangeBasis1D(order)
mesh  = Mesh1D(topo, basis)
mesh.build_mesh(vertices, elem_to_vertex, [0,-1])

cub_points, cub_weights = topo.get_quadrature(order+1)

In [ ]:
Kloc = np.zeros((basis.n_dofs, basis.n_dofs),
                dtype=np.double)
cub_vals = basis.eval_ref(np.eye(basis.n_dofs),
                          cub_points, d=1)

for i in range(basis.n_dofs):
    for j in range(basis.n_dofs):
        Kloc[i,j] = np.sum(cub_vals[i]*cub_vals[j]*cub_weights)
        
K = simple_assembly(mesh, Kloc)

def f(x):
    return x*x*(x-L)

def f2(x):
    return 6*x-2*L

rhs = simple_build_rhs(topo, basis, mesh, f2)

In [ ]:
sol = sps.linalg.spsolve(K, rhs)

In [ ]:
x_vals = np.linspace(0, L, mesh.n_elems+1)
x_vals = (x_vals[:-1]+x_vals[1:])/2.0
s = basis.eval_ref(sol[mesh.elem_to_dof], np.array([0.0])).ravel()

np.max(np.abs(s-f(x_vals)))

In [ ]:
plt.plot(x_vals, s)
plt.plot(x_vals, f(x_vals))

In [ ]:
plt.spy(K)